# Lesson 3: Building an Agent Reasoning Loop

## Setup

In [1]:
from helper import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()

In [2]:
import nest_asyncio
nest_asyncio.apply()

## Load the data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

## Setup the Query Tools

In [3]:
from utils import get_doc_tools

vector_tool, summary_tool = get_doc_tools("metagpt.pdf", "metagpt")

## Setup Function Calling Agent

In [4]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

In [5]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [6]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
In MetaGPT, the agent roles include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. The Product Manager is responsible for analyzing user requirements and creating a Product Requirement Document (PRD). The Architect translates these requirements into system design components. The Project Manager handles task distribution, while Engineers execute code based on the design. The QA Engineer formulates test cases to ensure code quality. Additionally, there are agents involved in reviewing feedback and adjusting constraint prompts to enhance learning and performance within the software development process.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "communication between agent 

In [7]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-05-09
last_modified_date: 2024-04-23

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
complex tasks, however, 

In [8]:
response = agent.chat(
    "Tell me about the evaluation datasets used."
)

Added user message to memory: Tell me about the evaluation datasets used.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "evaluation datasets in MetaGPT"}
=== Function Output ===
The evaluation datasets in MetaGPT include HumanEval, MBPP, and SoftwareDev. HumanEval comprises 164 handwritten programming tasks, MBPP includes 427 Python tasks, and SoftwareDev consists of 70 diverse software development tasks covering various scopes like mini-games, image processing algorithms, and data visualization. These datasets are utilized to assess MetaGPT's performance in code generation tasks and are compared against other benchmarks to demonstrate its capabilities.
=== LLM Response ===
The evaluation datasets used in MetaGPT include HumanEval, MBPP, and SoftwareDev. HumanEval consists of 164 handwritten programming tasks, MBPP includes 427 Python tasks, and SoftwareDev comprises 70 diverse software development tasks covering areas like mini-games, image proce

In [9]:
response = agent.chat("Tell me the results over one of the above datasets.")

Added user message to memory: Tell me the results over one of the above datasets.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "results over HumanEval dataset", "page_numbers": ["6"]}
=== Function Output ===
The results over the HumanEval dataset were utilized as one of the experimental settings in the study mentioned in the provided context.
=== LLM Response ===
The results over the HumanEval dataset were utilized as one of the experimental settings in the study mentioned in the provided context.


## Lower-Level: Debuggability and Control

In [10]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [11]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [12]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "agent roles in MetaGPT"}
=== Function Output ===
The agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. The Product Manager is responsible for generating the Product Requirement Document (PRD) outlining goals, user stories, competitive analysis, and requirement analysis. The Architect translates requirements into system design components. The Project Manager handles task distribution. Engineers execute designated classes and functions based on the design. The QA Engineer formulates test cases to ensure code quality. These roles work together sequentially following Standard Operating Procedures (SOPs) to efficiently complete complex software development tasks.


In [13]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task da5a3b1b-05f8-496c-b01a-438b19faac3a: 1
The agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. The Product Manager is responsible for generating the Product Requirement Document (PRD) outlining goals, user stories, competitive analysis, and requirement analysis. The Architect translates requirements into system design components. The Project Manager handles task distribution. Engineers execute designated classes and functions based on the design. The QA Engineer formulates test cases to ensure code quality. These roles work together sequentially following Standard Operating Procedures (SOPs) to efficiently complete complex software development tasks.


In [14]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task da5a3b1b-05f8-496c-b01a-438b19faac3a: 1


TaskStep(task_id='da5a3b1b-05f8-496c-b01a-438b19faac3a', step_id='9b396190-8c72-475e-bc81-96805cd77949', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [15]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

Added user message to memory: What about how agents share information?
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "how agents share information in MetaGPT"}
=== Function Output ===
Agents in MetaGPT share information through a structured communication protocol that includes a shared message pool where agents publish structured messages and subscribe to relevant messages based on their profiles. This protocol enhances role communication efficiency by allowing agents to obtain directional information from other roles and public information from the environment. Additionally, MetaGPT implements a publish-subscribe mechanism that enables agents to exchange information effectively within the multi-agent system. Agents also review previous feedback, make necessary adjustments to their constraint prompts, and utilize a handover feedback action to summarize information received during the development of previous projects. This information is stored in 

In [16]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

=== LLM Response ===
The agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. They communicate with each other through a structured communication protocol that includes a shared message pool, a publish-subscribe mechanism, and a structured workflow. This protocol allows agents to exchange information efficiently within the multi-agent system, review feedback, make adjustments, and summarize information for future projects. The roles collaborate sequentially to ensure the successful completion of software development tasks.
True


In [17]:
response = agent.finalize_response(task.task_id)

In [18]:
print(str(response))

assistant: The agent roles in MetaGPT include Product Manager, Architect, Project Manager, Engineer, and QA Engineer. They communicate with each other through a structured communication protocol that includes a shared message pool, a publish-subscribe mechanism, and a structured workflow. This protocol allows agents to exchange information efficiently within the multi-agent system, review feedback, make adjustments, and summarize information for future projects. The roles collaborate sequentially to ensure the successful completion of software development tasks.
